# Import and initialize

## Importing modules

In [1]:
%matplotlib notebook

from IPython.display import clear_output

import sys
sys.path.append(r"D:\OneDrive\BF3\Code");
sys.path.append(r"D:\OneDrive\BF3\Code\labtools-master")

import os
import time
import datetime
import numpy as np
import matplotlib as mpl
import time
# import utils

from collections import OrderedDict
from importlib import reload
from matplotlib import pyplot as plt

# from labtools import mplplots; reload(mplplots)
# from labtools.mplplots.init_nb_plotting import *
# from labtools.mplplots import tools as mpltools

import qcodes as qc
from qcodes.dataset.database import initialise_database, get_DB_location

#qc.config.subscription.default_subscribers  = ["Plottr"] # use plottr as the live plotting tool

from qcodes import new_experiment, Station, Instrument
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.plotting import plot_by_id
from qcodes.dataset.data_set import DataSet
from qcodes.dataset.data_set import load_by_id
from qcodes.dataset.data_export import get_data_by_id

from pytopo.qctools import instruments as instools

#from v2_dataset_tools.experiments import do_experiment, get_results_from_db_path

# from MDAC import MDAC 
# from qcodes.instrument_drivers.Keysight.Keysight_34465A import Keysight_34465A

#from qcodes.instrument_drivers.stanford_research.SR860 import SR860
#from qcodes.instrument_drivers.QuTech.IVVI import IVVI
#from qcodes.instrument_drivers.yokogawa.GS200 import GS200

from pytopo.sweep import sweep, measure, nest, chain, szip, getter, setter, time_trace, do_experiment
from pytopo.qctools.instruments import create_inst
from pytopo.qctools.dataset2 import select_experiment

from pytopo.qctools.instruments import create_inst
from pytopo.qctools.dataset2 import select_experiment
#qc.config.subscription.default_subscribers.pop()
qc.config.subscription.default_subscribers

[]

## Constants

In [2]:
e_ = 1.60217662e-19#C
h = 6.62607004e-34 #Js
G0 = 2*e_**2/h
Phi_0 = h/(2*e_)


## Database configuration

In [3]:
from qcodes.dataset.sqlite_base import transaction, one

def get_timestamp(run_id):
    DB = qc.config["core"]["db_location"]
    
    d = DataSet(DB)
    sql = """
    SELECT run_timestamp
    FROM
      runs
    WHERE
      run_id= ?
    """
    c = transaction(d.conn, sql, run_id)
    run_timestamp = one(c, 'run_timestamp')
    return run_timestamp

def timestamp_to_fmt(ts, fmt):
    return time.strftime(fmt, time.gmtime(ts))

def img_basepath(run_id):
    ts = get_timestamp(run_id)
    return timestamp_to_fmt(ts, qc.config['user']['img_dir'] + str(run_id).zfill(4) + '_')

def ds_title(run_id):
    return "{} #{}".format(os.path.abspath(qc.config['core']['db_location']), run_id)

qc.config['user']['img_dir'] = "d:/data/images/%Y-%m/%Y-%m-%d/"
qc.config.save_to_cwd()

In [4]:
#qc.config["core"]["db_location"] = datetime.date.today().strftime(r'D:\OneDrive\BF3\Data\experiments_%Y-%m-%d.db')
qc.config["core"]["db_location"] = datetime.date.today().strftime(r'D:\OneDrive\BF3\Data\experiments_2019-05-24.db')
#r'D:\OneDrive\BF3\Data\experiments_2019-03-21.db'
# automatically uses a different DB file for each month
initialise_database()
# creates a new DB file if nonexistant; leaves it untouched if file already exists

In [5]:
station.snapshot()

NameError: name 'station' is not defined

In [ ]:
ts = time.localtime()
pth = 'Z:\\'+'%02d-%02d-%02d'%(ts.tm_year-2000, ts.tm_mon, ts.tm_mday) + '\\CH9 T 19-03-26.log'
s = np.loadtxt(pth, usecols = (2), delimiter = ',')
print(s)

## Initialize instruments

### Lockins

In [6]:
station = Station()

from qcodes.instrument_drivers.Keysight.Keysight_34465A import Keysight_34465A
key1 = create_inst(Keysight_34465A, "key1", "USB0::0x2A8D::0x0301::MY57505052::INSTR", force_new_instance=True)
#key2 = create_inst(Keysight_34465A, "key2", "USB0::0x2A8D::0x0101::MY57503135::INSTR", force_new_instance=True)

from qcodes.instrument_drivers.stanford_research.SR865 import SR865
lockin1 = create_inst(SR865, "lockin1", "GPIB0::4::INSTR", force_new_instance=True)
lockin1.amplitude.vals._min_value = 0

lockin2 = create_inst(SR865, "lockin2", "GPIB0::5::INSTR", force_new_instance=True)
lockin2.amplitude.vals._min_value = 0
sys.path.append(r"D:\OneDrive\BF3\Code\StationQ\MDAC\Driver")

station = qc.Station(key1, lockin1, lockin2)

Connected to: Keysight Technologies 34465A (serial:MY57505052, firmware:A.02.16-02.40-02.16-00.51-03-01) in 0.95s
Connected to: Stanford_Research_Systems SR860 (serial:003225, firmware:V1.47) in 0.19s
Connected to: Stanford_Research_Systems SR860 (serial:003214, firmware:V1.47) in 0.07s


In [7]:
lockin3 = create_inst(SR865, "lockin3", "GPIB0::6::INSTR", force_new_instance=True)
lockin3.amplitude.vals._min_value = 0

Connected to: Stanford_Research_Systems SR860 (serial:003223, firmware:v1.51) in 0.07s


In [8]:
print('lockin1 excitation amplitude =', lockin1.amplitude(), 'V')
print('lockin1 DC bias =', lockin1.sine_outdc(), 'V')
print('lockin1 frequency =', lockin1.frequency(), 'Hz')
print('lockin1 phase =', lockin1.phase(), 'deg')
print('lockin1 X =', lockin1.X(), 'V')
print('lockin1 Y =', lockin1.Y(), 'V')
print('lockin1 R =', lockin1.R(), 'V')
print('lockin1 P =', lockin1.P(), 'deg')

lockin1 excitation amplitude = 0.0099999997765 V
lockin1 DC bias = 0.0 V
lockin1 frequency = 17.5806 Hz
lockin1 phase = 0.0 deg
lockin1 X = 1.8148033632e-05 V
lockin1 Y = -3.6210428789e-06 V
lockin1 R = 1.8505088519e-05 V
lockin1 P = -11.290825844 deg


### MDAC

In [9]:
import sys
sys.path.append(r"D:\OneDrive\BF3\Code");
sys.path.append(r"D:\OneDrive\BF3\Code\labtools-master")
sys.path.append(r"D:\OneDrive\BF3\Code\StationQ\MDAC\Driver")
from MDAC import MDAC
from pytopo.sweep import sweep, measure, nest, chain, szip, getter, setter, time_trace, do_experiment
from pytopo.qctools.instruments import create_inst


mdac = MDAC('MDAC', 'ASRL4::INSTR', debug=False, logging=False)

station.add_component(mdac)



C:\ProgramData\Anaconda3\lib\site-packages\pyvisa\ctwrapper\functions.py:1649: VisaIOWarning: VI_WARN_NSUP_ATTR_STATE (1073676420): Although the specified state of the attribute is valid, it is not supported by this implementation.
  return library.viSetAttribute(session, attribute, attribute_state)


Connected to: QNL Sydney MDAC (serial:0x32353532353551040028003E, firmware:1.4) in 2.94s


'MDAC'

### Custom instrument classes
#### Device class

In [10]:
import numpy as np
gate_ramp_stepsize = 1e-3  #[V] previously was 1e-3
gate_ramp_delay = 0.01  #[s]
def ramp(param, target, step_size, delay): 
    
    current_value = param()
    step_size *= np.sign(target - current_value)
    if abs(step_size) < 1E-10: 
        return 

    steps = np.arange(current_value, target, step_size)
    #print('sweep_time: %s s'%(delay*len(steps)))
    for step in steps: 
        param(step)
        time.sleep(delay)

    param(target)

    
def set_mdac_in_safemode(nchan=64):
    #ivvi.dac1(0)
    #ivvi.dac3(0)
    lockin1.amplitude(0)
    lockin1.sine_outdc(0)
    
    print('setting mdac in safe mode')
    
    for ch in mdac.channels[:nchan]:
        ramp(ch.voltage, 0, gate_ramp_stepsize, gate_ramp_delay)

    # Note: we do this in separate loops because we want to be sure that
    # no currents are running anywhere before switching the relays
    for ch in mdac.channels[:nchan]:
        ch.dac_output('open')
        ch.bus('open')
        ch.gnd('open')
        ch.smc('open')
        ch.microd('open')
        #ch.divider('off')
    print('Done')

def chs_safe(*channels):
    lockin1.amplitude(0)
    lockin1.sine_outdc(0)   
    print('setting mdac in safe mode')
    for ch in channels:
        ramp(ch.voltage, 0, gate_ramp_stepsize, gate_ramp_delay)
        ch.dac_output('open')
        ch.bus('open')
        ch.gnd('open')
        ch.smc('open')
        ch.microd('open')
    print('Done')

class Transistor(Instrument):
    '''general class for a gate tunable device'''
    def __init__(self, name, gate_dac, bias_dac, X, Y, bias_mode, **kw):
        super().__init__(name, **kw)   
        self._gate_dac = gate_dac
        self._bias_dac = bias_dac
        self.modes = {'Ibias':'A', 'Vbias':'V'}
        self.bias_mode = bias_mode
        self._X = X
        self._Y = Y
        self.Rbias = 1e6
        self.Ibias = bias_dac()/self.Rbias
        
        
        
        
        self.add_parameter('gate', get_cmd=self._gate_dac, set_cmd=self._ramp_gate, unit='V')
        self.add_parameter('bias', get_cmd=self._bias_dac, set_cmd=self._ramp_bias, unit=self.modes[bias_mode])
        self.add_parameter('susceptance', get_cmd=self._susceptance, set_cmd=None, unit=r'G/G0')
        self.add_parameter('conductance', get_cmd=self._conductance, set_cmd=None, unit=r'G/G0')
        self.add_parameter('admittance', get_cmd=self._admittance, set_cmd=None, unit=r'G/G0')
        self.add_parameter('impedance', get_cmd=self._impedance, set_cmd=None, unit=r'R/Rq')
        self.add_parameter('reactance', get_cmd=self._reactance, set_cmd=None, unit=r'R/Rq')
        self.add_parameter('resistance', get_cmd=self._resistance, set_cmd=None, unit=r'R/Rq')
        self._delay = 0.01        
        self._stepsize = 1e-3
    
    def _ramp_gate(self, val):
        ramp(self._gate_dac, val, self._stepsize, self._delay)
    def _ramp_bias(self, val):
        ramp(self._bias_dac, val, self._stepsize, self._delay)
    def _impedance(self):
        if self.bias_mode == 'Ibias':
            self.Ibias = self._bias_dac()/self.Rbias
            Vm = self._X() + 1.j*self._Y()
            Z = Vm/self.Ibias
            return G0*Z
        else:
            return 1./self.admittance()
    def _resistance(self):
        return np.real(self._impedance())
    def _reactance(self):
        return np.imag(self._impedance())
    
    def _admittance(self):
        if self.bias_mode == 'Vbias':
            Im = self.X() + 1.j*self.Y()
            return Im/self._bias_dac()/G0
        else:
            return 1./self._impedance()
    def _conductance(self):
        Y = self._admittance()
        return np.real(Y)
    def _susceptance(self):
        Y = self._admittance()
        return np.imag(Y)
    
    
#HB2 = Transistor('HB2_Rbend2', mdac.ch01.voltage, lockin1.amplitude, lockin1.X, lockin1.Y, 'Ibias')
RB = instools.create_inst(Transistor, 'HB2_Rbend', *(mdac.ch01.voltage, lockin1.amplitude, lockin1.X, lockin1.Y, 'Ibias'), force_new_instance = True)
RH = instools.create_inst(Transistor, 'HB2_RHall', *(mdac.ch01.voltage, lockin1.amplitude, lockin2.X, lockin2.Y, 'Ibias'), force_new_instance = True)
Rxx = instools.create_inst(Transistor, 'HB2_Rxx', *(mdac.ch01.voltage, lockin1.amplitude, lockin3.X, lockin3.Y, 'Ibias'), force_new_instance = True)

In [11]:
RB.impedance()/G0
RH.impedance()/G0
Rxx.impedance()/G0
RB.gate(0.40)

#### R vs T instrument

In [12]:
class ProbeThermometer(Instrument):
    
    def __init__(self, name, **kw):
        super().__init__(name, **kw)        
        self.add_parameter('temperature', get_cmd=self._get_temperature, set_cmd=None, unit='K')
        self._ind = 0
        
    
    def _get_temperature(self):
        ts = time.localtime()
        ds = '%02d-%02d-%02d'%(ts.tm_year-2000, ts.tm_mon, ts.tm_mday)
        pth = r'Z:\\'+ ds + '\\CH9 T '+ds+'.log'
        time.sleep(1)
        s = np.loadtxt(pth, usecols = (2), delimiter = ',')
        return s[-1]
    
        
    
T_probe = create_inst(ProbeThermometer, 'Tprobe', force_new_instance=True)




In [25]:
set_mdac_in_safemode()

setting mdac in safe mode
Done


### Heater

In [ ]:
#from front panel - interface (6) - select connection to ethernet
from qcodes.instrument_drivers.Lakeshore import Model_372
ls = instools.create_inst(Model_372.Model_372, 'lakeshore_372', 'TCPIP::192.168.0.160::7777::SOCKET')
h = ls.sample_heater

In [ ]:
#Example for heater settings (change it back to USB mode from lakeshore front panel to see it from the PC)

# Specify heater channel
heater_channel = 9;

# Set auto range of heater to enabled = 'current'
heater = ls.channels[heater_channel-1];
heater.auto_range('current')

# PID setup
h.P(1)
h.I(2)
h.D(2)
h.input_channel(heater_channel)
h.setpoint(0) # initialize setpoint to 0 for safety
h.mode('closed_loop')
max_T = [0.014, 0.019, 0.05, 0.15, 0.49, 1.5, 4.5, 10];
h.range_limits(max_T)

### Magnet
Check that the magnet is the "right" one and so the parameters (coil constant and so on...)

In [14]:
from qcodes.instrument_drivers.american_magnetics.AMI430 import AMI430,AMI430_3D
ami_x = instools.create_inst(AMI430, "AMI430_x", "169.254.178.28", port = 7180, has_current_rating=True)
ami_y = instools.create_inst(AMI430, "AMI430_y", "169.254.175.39", port = 7180, has_current_rating=True)
ami_z = instools.create_inst(AMI430, "AMI430_z", "169.254.137.107", port = 7180, has_current_rating=True)
ami = instools.create_inst(AMI430_3D, "AMI430", ami_x, ami_y, ami_z, 2)
ami_3d = AMI430_3D("AMI_3D", ami_x, ami_y, ami_z, 1);

Connected to: AMERICAN MAGNETICS INC. 430 (serial:170029, firmware:2.59) in 1.40s
Connected to: AMERICAN MAGNETICS INC. 430 (serial:170028, firmware:2.59) in 0.95s
Connected to: AMERICAN MAGNETICS INC. 430 (serial:170027, firmware:2.59) in 1.14s


In [ ]:



ramp_limit_multiplier = 0.6
ami_x.current_ramp_limit(0.1)
ami_x.coil_constant(0.01466)
ami_x.current_rating(68.21)

ami_y.current_ramp_limit(0.1)
ami_y.coil_constant(0.01464)
ami_y.current_rating(68.31)

ami_z.current_ramp_limit(0.0775*ramp_limit_multiplier)
ami_z.coil_constant(0.0818)
ami_z.current_rating(73.35)



In [ ]:
print(ami_x.ramp_rate())
print(ami_y.ramp_rate())
print(ami_z.ramp_rate())
ami_x.current_ramp_limit
ami_z.current_ramp_limit()
#30.1802*ramp_limit_multiplier

In [ ]:
print(ami_x.field())
print(ami_y.field())
print(ami_z.field())
print (ami_3d.field_measured())
ami_3d.phi_measured()
ami_x.field(0.01)



#### Example for magnet non-blocking sweep (ramp_to is the non blocking funcion instead of field function)

In [ ]:
x_setpoint = 0 #E-3; # T
y_setpoint = 0 #E-3; # T
z_setpoint = 0 #E-3; # T

fig = plt.figure('set B');
fig.clear();
fpl = fig.add_subplot(1, 1, 1);

Bx, By, Bz = [], [], [];
eps = 1E-5; # T

ami_x.ramp_rate(0.001); # T/s
ami_y.ramp_rate(0.001); # T/s
ami_z.ramp_rate(0.0015); # T/s

ami_x.ramp_to(x_setpoint);
ami_y.ramp_to(y_setpoint);
ami_z.ramp_to(z_setpoint);

while np.abs(x_setpoint - ami_x.field()) > eps or np.abs(y_setpoint - ami_y.field()) > eps or np.abs(z_setpoint - ami_z.field()) > eps:
    
    Bx.append(ami_x.field());
    By.append(ami_y.field());
    Bz.append(ami_z.field());
    
    fpl.clear();
    fpl.plot(range(0, len(Bx)), np.array(Bx) * 1E3, range(0, len(By)), np.array(By)*1E3, range(0, len(Bz)), np.array(Bz)*1E3);
    fpl.legend(('$B_x$ = {} mT'.format(str(round(Bx[-1]*1E3, 3))), '$B_y$ = {} mT'.format(str(round(By[-1]*1E3, 3))), '$B_z$ = {} mT'.format(str(round(Bz[-1]*1E3, 3)))));
    plt.xlabel('time (s)');
    plt.ylabel('$B_{x,y,z}$ (mT)');
    fig.canvas.draw();
    
    time.sleep(1);

In [ ]:
ami_x.ramp_to(0)
ami_y.ramp_to(0)
ami_z.ramp_to(0)

#### 3D magnet setup

### Playground
![title](240px-3D_Spherical.svg.png)

In [ ]:
# Block/non-block toggle
ami_3d.block_during_ramp(not ami_3d.block_during_ramp())
ami_3d.block_during_ramp()

In [ ]:
# (r, theta, phi)
ami_3d.spherical((0.01, 0, 90))

In [ ]:
ami_3d.spherical_measured()

In [ ]:
ami_3d.cartesian_measured()
ami_3d.block_during_ramp(False)

## Measurements
### Measurement functions
The one for setting lockin voltage should be the only one. If not check the mdac ones in Q216F notebook

In [ ]:
@setter(("lockin_dc", "V"))
def lockin_dc(voltage): 
    voltage *= 100  # Takes into account the attenuator at lockin DC output
    ramp(lockin1.sine_outdc, voltage, bias_ramp_stepsize, bias_ramp_delay)

### Resistance

In [ ]:
def setup():
    key1.NPLC(1)
    lockin_dc(10e-3)
    lockin1.amplitude(5e-4)     # 1e-2 attenuation so 10uV excitation when this is 1mV
    #sr1.frequency(117)         # adder HPF cutoff is 15Hz
    #sr1.phase(0)
    #sr1.time_constant(.1)
    #ami_x.field(0)
    #ami_x.ramp_to(-0.2)
    time.sleep(3)

def cleanup(): 
    lockin1.amplitude(0)
    lockin_dc(0)
    #ami_x.field(0)
    #ami_z.field(0)

@getter(('current', 'A'), ('raw_voltage_dc', 'V'), ('conductance', '2e^2/h'),
        ('Lockin1_X', 'V'), ('Lockin1_Y', 'V'), ('Lockin1_R', 'V'),
        ('Lockin1_phase', 'deg'), ('Lockin1_excitation', 'V'))
def measure_acdc(gain=1e6, Rseries=13e3, adder_atten=1e-2):
    time.sleep(0.08)
    #B_x = ami_x.field()
    #V_meas = key2.volt()
    if np.abs(np.abs(lockin1.sine_outdc())-0.05) < 2e-3: # make this range larger than distance btw two points
        time.sleep(3)
    key_raw = key1.volt()
    current_dc = -key_raw/gain
    Vac     = adder_atten*lockin1.amplitude()  #*np.sqrt(2)
    L1P     = lockin1.P() # in this order because this gives lockin_x more time to stailize and allows a shorter sleeping time
    L1R     = lockin1.R()
    L1Y     = lockin1.Y()
    # the above lines (excluding time.sleep) take at least 75ms to execute. 
    L1X     = lockin1.X()
    current_ac = L1X/gain    
    try:
        g = 12906/(Vac/current_ac - Rseries) #[2e^2/h]
    except ZeroDivisionError:
        g = 0

    result  = current_dc, key_raw, g, L1X, L1Y, L1R, L1P, Vac
    return result

result = do_experiment(
    "Experiment_name/Sample_name",
    nest(
        #sweep(mdac_PG_voltage, np.linspace(1.6, 1.7, 101)),
        sweep(lockin_dc, np.linspace(-1e-3, 1e-3, 201)), # real voltage applied onto device
        measure(measure_acdc)
    ),
    setup,
    cleanup,
    live_plot=True
)

### R vs T

In [ ]:
def setup():
    
    lockin1.sine_outdc(0)
    lockin1.amplitude(2)
    lockin2.sine_outdc(0)
    lockin2.amplitude(2)
    time.sleep(1)
    
def cleanup(): 
    lockin1.sine_outdc(0)
    lockin1.amplitude(0)
    lockin2.sine_outdc(0)
    lockin2.amplitude(0)


    

setup()
temp = meas_ins.temperature()
r1 = meas_ins.R_UCSB_dev3()
r2 = meas_ins.R_UCSB_dev4()

print('temp =', temp, 'K')  
print('R1 =', r1, 'Ohms') 
print('R2 =', r2, 'Ohms')

type(meas_ins.temperature())



In [ ]:
meas_ins.temperature()

In [ ]:
@setter(("time", "a.u."))
def time_sweep(t): 
    pass

In [ ]:
def setup():
    
    lockin1.sine_outdc(0)
    lockin1.amplitude(2)
    lockin2.sine_outdc(0)
    lockin2.amplitude(2)
    time.sleep(1)

def cleanup(): 
    lockin1.sine_outdc(0)
    lockin1.amplitude(0)
    lockin2.sine_outdc(0)
    lockin2.amplitude(0)


@getter(('Temperature', 'K'),
        ('R_1_real', 'Ohm'), ('R_1_imag', 'Ohm'), ('R_2_real', 'Ohm'), ('R_2_imag', 'Ohm'),
        ('Lockin1_R', 'V'), ('Lockin1_P', 'deg'), ('Lockin2_R', 'V'), ('Lockin2_P', 'deg'),
        ('I_exc_1', 'A'), ('I_exc_2', 'A'))
def measure_ac_Ibias(R_bias=1e6):
    time.sleep(5)
    
    T = meas_ins.temperature()

    Iac1     = lockin1.amplitude()/R_bias # excitation current  in A
    L1P     = lockin1.P() # in this order because this gives lockin_x more time to stailize and allows a shorter sleeping time
    L1R     = lockin1.R()
    L1Y     = lockin1.Y()
    # the above lines (excluding time.sleep) take at least 75ms to execute. 
    L1X     = lockin1.X()
    Rac1_real = L1X/Iac1    
    Rac1_imag = L1Y/Iac1
    
    Iac2     = lockin2.amplitude()/R_bias # excitation current in A
    L2P     = lockin2.P() # in this order because this gives lockin_x more time to stailize and allows a shorter sleeping time
    L2R     = lockin2.R()
    L2Y     = lockin2.Y()
    # the above lines (excluding time.sleep) take at least 75ms to execute. 
    L2X     = lockin2.X()
    Rac2_real = L2X/Iac2  
    Rac2_imag = L2Y/Iac2

    result  = T, Rac1_real, Rac1_imag, Rac2_real, Rac2_imag, L1R, L1P, L2R, L2P, Iac1, Iac2
    return result

result = do_experiment(
    "R_Al_film_cooldown/UCSB_dev4",
    nest(
        #sweep(mdac_PG_voltage, np.linspace(1.6, 1.7, 101)),
        sweep(time_sweep, np.linspace(0, 1, 5001)),
        measure(measure_ac_Ibias)
    ),
    setup,
    cleanup,
    live_plot=True
)

In [ ]:
import sys
exp = select_experiment('Al_Characterizations', 'MBE_Al_TUe_Al')

meas = Measurement(exp=exp)
meas.register_parameter(meas_ins.counter)
meas.register_parameter(meas_ins.temperature, setpoints=[meas_ins.counter])
meas.register_parameter(meas_ins.R_MBE, setpoints=[meas_ins.counter])
meas.register_parameter(meas_ins.R_MOVPE, setpoints=[meas_ins.counter])
setup()


with meas.run() as datasaver:

    

    #datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
    #                            min_wait=0, min_count=1)
    kk = 0
    while kk<1000:# meas_ins.temperature() > 0.0.: 
        sys.stdout.write('\b\b\b\b%s'%kk)
        datasaver.add_result((meas_ins.counter, meas_ins.counter()),
                            (meas_ins.temperature, meas_ins.temperature()),
                            (meas_ins.R_MBE, meas_ins.R_MBE()),
                            (meas_ins.R_MOVPE, meas_ins.R_MOVPE()))
        kk+=1
cleanup()

In [ ]:
setup()


#### Plotting traces


In [ ]:
dat_cooldown = load_by_id(3)

In [ ]:
dbpth = 'experiments_2019-03-26.db'
dat= load_by_id(3)
pars = dat.get_parameters()
c = pars[0]
T = pars[1]
RMBE = pars[2]
RMOVPE = pars[3]
ind = 300
cs = np.array(dat.get_values(c.name))[:ind,0]*2/60
Ts = np.array(dat.get_values(T.name))[:ind,0]
RMBE = np.array(dat.get_values(RMBE.name))[:ind,0]*np.pi/np.log(2)
RMOVPE = np.array(dat.get_values(RMOVPE.name))[:ind,0]*np.pi/np.log(2)
fig, ax1 = plt.subplots()
ax1.set_title('experiments_2019-03-21.db: runid_1')

ax1.plot(cs, RMBE,'-o',label = 'DMBE')
ax1.plot(cs, RMOVPE,'-o',label = 'UCSB')
plt.ylabel(r'$R_\mathrm{S}$ ($\Omega/$sq)')
ax2 = ax1.twinx()
sel = np.abs(np.diff(Ts))>0.01
ax2.plot(cs[:-1][sel], Ts[:-1][sel], 'go-')
ax2.tick_params(axis='y', labelcolor='g')
ax2.set_ylabel(r'$T_\mathrm{probe}$ (K)', color = 'g', rotation = -90,labelpad=20)#, pad = 30 )
#ax2.plot(np.diff(Ts))
ax1.legend()
plt.tight_layout()
ax1.set_xlabel('time (min.)')

In [ ]:
plottr

In [ ]:
from plottr import client

In [ ]:
meas.register_custom_parameter?


In [ ]:
np.sqrt(1.268**2+0.037**2)

### Hall bars
#### quickscan pinchoff

In [ ]:
mdac.ch01.smc('close')
mdac.ch01.dac_output('close')
mdac.ch01.rate(0.2)
mdac.ch01.ramp(0.0)
#lockin1.amplitude(Ib*Rin)

In [ ]:
mdac.ch01.voltage()

In [ ]:
mdac.ch01.rate(0.2)
mdac.ch02.ramp(0.0)
Rin = 1e6
Ib = 1e-9
lockin1.amplitude(Ib*Rin)

In [ ]:
HB2.gate(0.4)

##### Impedance

In [ ]:
Rin = 1e6
Ib = 10e-9
HB2.bias(Rin*Ib)
HB2.gate(0.0)

print(HB2.impedance())
print(HB2.admittance())

Vgs = np.linspace(0,1, 21)
HB2.gate(Vgs[0])
time.sleep(0.5)
Zs = []
Rs = []
Xs = []
for Vg in Vgs:
    HB2.gate(Vg)
    time.sleep(0.1)
    Zs += [np.abs(HB2.impedance())]
    Rs += [HB2.resistance()]
    Xs += [HB2.reactance()]
    
HB2.gate(0)
plt.plot(Vgs, np.array(Zs), label = '|Z|')
plt.plot(Vgs, np.array(Rs), label = 'R')
plt.plot(Vgs, np.array(Xs), label = 'X')
plt.legend()

##### Admittance

In [ ]:
Rin = 1e6
Ib = 10e-9
HB2.bias(Rin*Ib)
HB2.gate(0.0)

print(HB2.impedance())
print(HB2.admittance())

Vgs = np.linspace(0,1, 21)
Vgs = np.append(Vgs, Vgs[::-1])
HB2.gate(Vgs[0])
time.sleep(0.5)
Ys = []
Gs = []
Ss = []
for Vg in Vgs:
    HB2.gate(Vg)
    time.sleep(0.1)
    Ys += [np.abs(HB2.admittance())]
    Gs += [HB2.conductance()]
    Ss += [HB2.susceptance()]
    
HB2.gate(0)
plt.plot(Vgs, np.array(Ys), label = 'Y')
plt.plot(Vgs, np.array(Gs), label = 'G')
plt.plot(Vgs, np.array(Ss), label = 'S')
plt.legend()

## Field measurements

In [ ]:
import sys
exp = select_experiment('Semiconductor_characterizations', 'Q243_HB2_Bend_resistance')

meas = Measurement(exp=exp)
meas.register_parameter()
meas.register_parameter(ami_x.field, setpoints=[meas_ins.counter])
meas.register_parameter(HB2.conductance, setpoints=[ami_x.field])
meas.register_parameter(HB2.susceptance, setpoints=[ami_x.field])



with meas.run() as datasaver:

    kk = 0
    while kk<1000:# meas_ins.temperature() > 0.0.: 
        sys.stdout.write('\b\b\b\b%s'%kk)
        datasaver.add_result((meas_ins.counter, meas_ins.counter()),
                            (meas_ins.temperature, meas_ins.temperature()),
                            (meas_ins.R_MBE, meas_ins.R_MBE()),
                            (meas_ins.R_MOVPE, meas_ins.R_MOVPE()))
        kk+=1
cleanup()

### Bend reistance
#### R_b(Bx) and RHall(Bx) 
RHall(Bx) = Rxx+Rxy because measured over crossed voltage probes

In [17]:
import sys
RB.gate(0.4)

exp = select_experiment('Semiconductor_characterizations', 'Q243_HB2_R_bend_IC7C9_VC11C10_R_Hall')

meas = Measurement(exp=exp)
fieldsetpoint = ami_z.field
meas.register_parameter(ami_z.field)
meas.register_parameter(RB.gate)

meas.register_custom_parameter('Rb_forward', unit = 'R/Rq', setpoints=[fieldsetpoint, RB.gate])
meas.register_custom_parameter('Rb_backward', unit = 'R/Rq', setpoints=[fieldsetpoint, RB.gate])
meas.register_custom_parameter('Xb_forward', unit = 'R/Rq', setpoints=[fieldsetpoint, RB.gate])
meas.register_custom_parameter('Xb_backward', unit = 'R/Rq', setpoints=[fieldsetpoint, RB.gate])
meas.register_custom_parameter('RH_forward', unit = 'R/Rq', setpoints=[fieldsetpoint, RB.gate])
meas.register_custom_parameter('RH_backward', unit = 'R/Rq', setpoints=[fieldsetpoint, RB.gate])
meas.register_custom_parameter('XH_forward', unit = 'R/Rq', setpoints=[fieldsetpoint, RB.gate])
meas.register_custom_parameter('XH_backward', unit = 'R/Rq', setpoints=[fieldsetpoint, RB.gate])
meas.register_custom_parameter('Rxx_forward', unit = 'R/Rq', setpoints=[fieldsetpoint, RB.gate])
meas.register_custom_parameter('Rxx_backward', unit = 'R/Rq', setpoints=[fieldsetpoint, RB.gate])
meas.register_custom_parameter('Xxx_forward', unit = 'R/Rq', setpoints=[fieldsetpoint, RB.gate])
meas.register_custom_parameter('Xxx_backward', unit = 'R/Rq', setpoints=[fieldsetpoint, RB.gate])
meas.register_custom_parameter('T_probe', unit='K', setpoints=[fieldsetpoint, RB.gate])
meas.register_custom_parameter('actual_field', unit = 'T', setpoints=[fieldsetpoint, RB.gate])

Vgs = np.linspace(0.4, 0.6, 101)
Bxs = np.linspace(0, 0.25, 126)


sweepdirs = [1,-1,1,-1]
fielddirs = [1,1,-1,-1]
kk = 0
for sweepdir, fielddir in zip(sweepdirs, fielddirs ):

    with meas.run() as datasaver:


        for Bx in fielddir*Bxs[::sweepdir]:
            current_field = ami_z.field()
            delay = np.abs((Bx-current_field)/0.0002)
            ami_z.set_field(Bx)
            
            #time.sleep(delay)
            #datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
            #                            min_wait=0, min_count=1)



            #sys.stdout.write(100*'\b')
            imps_forward = []
            imps_backward = []
            Hall_imps_forward = []
            Hall_imps_backward = []
            xx_imps_forward = []
            xx_imps_backward = []
            
            for Vg in Vgs:
                RB.gate(Vg)
                time.sleep(0.1)
                imps_forward += [RB.impedance()]
                Hall_imps_forward += [RH.impedance()]
                xx_imps_forward += [Rxx.impedance()]
            for Vg in Vgs[::-1]:
                RB.gate(Vg)
                time.sleep(0.1)
                imps_backward += [RB.impedance()]
                Hall_imps_backward += [RH.impedance()]
                xx_imps_backward += [Rxx.impedance()]
                
            T_p = T_probe.temperature()
            a_f= np.array(imps_forward) 
            a_b= np.array(imps_backward)[::-1]
            b_f= np.array(Hall_imps_forward) 
            b_b= np.array(Hall_imps_backward)[::-1]
            c_f= np.array(xx_imps_forward) 
            c_b= np.array(xx_imps_backward)[::-1]
            print('field: %sT'%Bx)   
            print('T_probe: %sK'%T_probe.temperature())  
            datasaver.add_result((ami_z.field, Bx),
                                 (RB.gate, Vgs),
                                ('Rb_forward', np.real(a_f)),
                                 ('Rb_backward', np.real(a_b)),
                                 ('Xb_forward', np.imag(a_f)),
                                 ('Xb_backward', np.imag(a_b)),
                                 ('RH_forward', np.real(b_f)),
                                 ('RH_backward', np.real(b_b)),
                                 ('XH_forward', np.imag(b_f)),
                                 ('XH_backward', np.imag(b_b)),
                                 ('Rxx_forward', np.real(c_f)),
                                 ('Rxx_backward', np.real(c_b)),
                                 ('Xxx_forward', np.imag(c_f)),
                                 ('Xxx_backward', np.imag(c_b)),
                                 ('T_probe', T_p),
                                 ('actual_field', ami_z.field())
                                )
                                
            kk+=1


Starting experimental run with id: 65
field: 0.0T
T_probe: 0.0742405K
field: 0.002T
T_probe: 0.0745496K
field: 0.004T
T_probe: 0.0746933K
field: 0.006T
T_probe: 0.0748332K
field: 0.008T
T_probe: 0.0750724K
field: 0.01T
T_probe: 0.0752635K
field: 0.012T
T_probe: 0.0752696K
field: 0.014T
T_probe: 0.0753651K
field: 0.016T
T_probe: 0.0752691K
field: 0.018T
T_probe: 0.0752629K
field: 0.02T
T_probe: 0.07523K
field: 0.022T
T_probe: 0.0752888K
field: 0.024T
T_probe: 0.075191K
field: 0.026T
T_probe: 0.0752476K
field: 0.028T
T_probe: 0.0752484K
field: 0.03T
T_probe: 0.0752869K
field: 0.032T
T_probe: 0.0752638K
field: 0.034T
T_probe: 0.0752993K
field: 0.036T
T_probe: 0.0752828K
field: 0.038T
T_probe: 0.0751852K
field: 0.04T
T_probe: 0.0752565K
field: 0.042T
T_probe: 0.075245K
field: 0.044T
T_probe: 0.0752056K
field: 0.046T
T_probe: 0.0752102K
field: 0.048T
T_probe: 0.0751684K
field: 0.05T
T_probe: 0.0752251K
field: 0.052T
T_probe: 0.075154K
field: 0.054T
T_probe: 0.0752092K
field: 0.056T
T_probe:

field: 0.02T
T_probe: 0.0745685K
field: 0.018T
T_probe: 0.0746786K
field: 0.016T
T_probe: 0.0746798K
field: 0.014T
T_probe: 0.0747434K
field: 0.012T
T_probe: 0.0750284K
field: 0.01T
T_probe: 0.0749468K
field: 0.008T
T_probe: 0.0749485K
field: 0.006T
T_probe: 0.075076K
field: 0.004T
T_probe: 0.0752134K
field: 0.002T
T_probe: 0.0752272K
field: 0.0T
T_probe: 0.0753418K
Starting experimental run with id: 67
field: -0.0T
T_probe: 0.0749688K
field: -0.002T
T_probe: 0.0753693K
field: -0.004T
T_probe: 0.0758428K
field: -0.006T
T_probe: 0.0761332K
field: -0.008T
T_probe: 0.0762791K
field: -0.01T
T_probe: 0.0763835K
field: -0.012T
T_probe: 0.0765133K
field: -0.014T
T_probe: 0.0766132K
field: -0.016T
T_probe: 0.0764695K
field: -0.018T
T_probe: 0.0763919K
field: -0.02T
T_probe: 0.0762762K
field: -0.022T
T_probe: 0.0763042K
field: -0.024T
T_probe: 0.0762101K
field: -0.026T
T_probe: 0.0762023K
field: -0.028T
T_probe: 0.0764815K
field: -0.03T
T_probe: 0.0762646K
field: -0.032T
T_probe: 0.0761637K
fie

T_probe: 0.0745548K
field: -0.054T
T_probe: 0.0745666K
field: -0.052T
T_probe: 0.0745692K
field: -0.05T
T_probe: 0.0746212K
field: -0.048T
T_probe: 0.0746051K
field: -0.046T
T_probe: 0.0746167K
field: -0.044T
T_probe: 0.0745976K
field: -0.042T
T_probe: 0.0745243K
field: -0.04T
T_probe: 0.0745059K
field: -0.038T
T_probe: 0.0745655K
field: -0.036T
T_probe: 0.0745619K
field: -0.034T
T_probe: 0.0745657K
field: -0.032T
T_probe: 0.0745931K
field: -0.03T
T_probe: 0.0746382K
field: -0.028T
T_probe: 0.0746482K
field: -0.026T
T_probe: 0.0746344K
field: -0.024T
T_probe: 0.0746314K
field: -0.022T
T_probe: 0.0746826K
field: -0.02T
T_probe: 0.0747273K
field: -0.018T
T_probe: 0.07478K
field: -0.016T
T_probe: 0.0747077K
field: -0.014T
T_probe: 0.0750899K
field: -0.012T
T_probe: 0.0750093K
field: -0.01T
T_probe: 0.074966K
field: -0.008T
T_probe: 0.0750211K
field: -0.006T
T_probe: 0.0751184K
field: -0.004T
T_probe: 0.0752065K
field: -0.002T
T_probe: 0.0753905K
field: -0.0T
T_probe: 0.0754741K


In [ ]:
len(b_f)

#### R_b(Bx) @ large Z-field

In [ ]:
import sys
ami_z.set_field(2)
time.sleep(300)
HB2.gate(0.4)

exp = select_experiment('Semiconductor_characterizations', 'Q243_HB2_Bend_resistance_IC7C9_VC11C10')

meas = Measurement(exp=exp)

meas.register_parameter(ami_x.field)
meas.register_parameter(HB2.gate)

meas.register_custom_parameter('resistance_forward', unit = 'R/Rq', setpoints=[ami_x.field, HB2.gate])
meas.register_custom_parameter('resistance_backward', unit = 'R/Rq', setpoints=[ami_x.field, HB2.gate])
meas.register_custom_parameter('reactance_forward', unit = 'R/Rq', setpoints=[ami_x.field, HB2.gate])
meas.register_custom_parameter('reactance_backward', unit = 'R/Rq', setpoints=[ami_x.field, HB2.gate])
meas.register_custom_parameter('Bx_measured', unit = 'T', setpoints=[ami_x.field, HB2.gate])
meas.register_custom_parameter('Bz_measured', unit = 'T', setpoints=[ami_x.field, HB2.gate])

Vgs = np.linspace(0.4, 0.6, 101)
Bxs = np.linspace(0,0.9,91)

sweepdirs = [1,-1,1,-1]
fielddirs = [1,1,-1,-1]
kk = 0
for sweepdir, fielddir in zip(sweepdirs, fielddirs ):

    with meas.run() as datasaver:


        for Bx in fielddir*Bxs[::sweepdir]:
            current_field = ami_x.field()
            delay = np.abs((Bx-current_field)/0.0002)
            ami_x.set_field(Bx)
            
            #time.sleep(delay)
            #datasaver.dataset.subscribe(QcodesDatasetSubscriber(datasaver.dataset), state=[], 
            #                            min_wait=0, min_count=1)



            #sys.stdout.write(100*'\b')
            imps_forward = []
            imps_backward = []
            for Vg in Vgs:
                HB2.gate(Vg)
                time.sleep(0.1)
                imps_forward += [HB2.impedance()]
            for Vg in Vgs[::-1]:
                HB2.gate(Vg)
                time.sleep(0.1)
                imps_backward += [HB2.impedance()]
            a_f= np.array(imps_forward) 
            a_b= np.array(imps_backward)[::-1]
            print('field: %sT'%Bx)   
            datasaver.add_result((ami_x.field, Bx),
                                 (HB2.gate, Vgs),
                                ('resistance_forward', np.real(a_f)),
                                 ('resistance_backward', np.real(a_b)),
                                 ('reactance_forward', np.imag(a_f)),
                                 ('reactance_backward', np.imag(a_b)),                                 
                                 ('Bx_measured', ami_x.field()),
                                 ('Bz_measured', ami_z.field()),
                                )
                                
            kk+=1


In [23]:
RB.gate(0.)
lockin1.amplitude(0)
ami_y.field()

-4.7e-06

In [ ]:
dbpth = 'experiments_2019-03-21.db'
dat= load_by_id(10)
pars = dat.get_parameters()
Ts=pars[0]
Bxs = pars[1]
R_MBEs = pars[7]
R_MOVPEs = pars[8]

np.array(dat.get_values(Ts.name))

plt.plot(np.array(dat.get_values(Bxs.name)), np.array(dat.get_values(R_MBEs.name)), 'o-')
plt.plot(np.array(dat.get_values(Bxs.name)), np.array(dat.get_values(R_MOVPEs.name)), 'o-')

In [ ]:
def _Bx(Bz):
    return 0.037/1.268*Bz

Bzs = np.linspace(0.8,2,41)
Bxs = _Bx(Bzs)

In [ ]:
ami_3d.x(0.02334385)

In [ ]:
ami_3d.x_measured()

In [ ]:
Bzs = np.linspace(0.8,2,121)#np.linspace(0.8,2,121)
121*(np.abs(np.diff(Bzs)[0])/(0.082*0.03)+2)


14*121

In [ ]:
0.03/0.00246

In [ ]:
np.diff(Bzs)[0]

In [ ]:
Bzs = np.linspace(0.8,2,121)
Bxs = _Bx(Bzs)
kk=0
for Bx, Bz in zip(Bxs, Bzs):
    print(Bx, Bz)

In [ ]:
ami_z.field(5.5)

In [ ]:
901*4*4/3600